In [1]:
import numpy as np
import pandas as pd
import re
import random
import time

NPD = pd.read_excel("NPD_Data.xlsx", header=0).values
amazon = pd.read_excel("Amazon_Unmapped4.xlsx", header=0).values

In [2]:
# Divides mapped from unmapped
# NPD_unmapped = NPD[NPD[:,4] == "Unmapped"]
amazon_mapped = amazon[amazon[:,3] == "Unmapped"]
NPD_mapped = NPD

In [3]:
# Divides description of each product into individual words
def splitter(df):
    split = [0]*df.shape[0]
    # Useless strings to be removed
    purge_list = ["","a","and","with","for","in","the","of","to","x","disney","-","&"]
    
    # Divides description into individual strings
    for i in range(0,df.shape[0]):
        split[i] = [x.lower() for x in (re.split('[ _,+\'/#?:;()|]',str(df[i][0])))]
        
        for j in purge_list:
            while(j in split[i]): 
                split[i].remove(j)
    return split

In [4]:
mNPDSplit = splitter(NPD_mapped)
mAmaSplit = splitter(amazon_mapped)

In [5]:
# Not super useful, just counts occurences of each term in the entire sheet
def count_occurrences(df):
    dict = {}
    for i in range(0,len(df)):
        for j in range(0,len(df[i])):
            if df[i][j] not in dict:
                dict[df[i][j]] = {}
            if i not in dict[df[i][j]]:
                dict[df[i][j]][i] = 0
            dict[df[i][j]][i] += 1
    return dict

In [6]:
train_NPD = count_occurrences(mNPDSplit)

In [7]:
# Finds occurences of each property, e.g. "Frozen": 2000
def count_properties(dic):
    count = {}
    for i in dic:
        count[i] = 0
        for j in dic[i]:
            count[i] += dic[i][j]
    return count

In [8]:
# # Literally no clue what this does, maybe the same as above but for a pandas dataframe
# count = {}
# for i in range(NPD_mapped.shape[0]):
#     if NPD_mapped[i][4] not in count:
#         count[NPD_mapped[i][4]] = 0
#     else:
#         count[NPD_mapped[i][4]] += 1
# count = {k: v for k, v in sorted(count.items(), key=lambda item: item[1], reverse=True)}

In [9]:
CP = count_properties(train_NPD)
# Sorts dictionary of properties into order of occurrence
CP = {k: v for k, v in sorted(CP.items(), key=lambda item: item[1], reverse=True)}

In [10]:
# Lists to be used for training
X = NPD_mapped[:,1]
y = mNPDSplit
# ama_prop = amazon_mapped[:,3]

In [11]:
# This will take FOREVER, so just let it sit and run for a good 20mins(?)
count = 0
X_dict = {}
# Go through all the properties and the list of products and add each indivdual word of item description to the property it associates with and tally the occurences of this word
# Word will then build an "identity of words that are commonly used with it.
load = len(CP)
prev = 0
start_time = time.time()
for listKeyword in CP:
    if round((count/load)*10) != prev:
            print(round((count/load)*10)*10, "% complete...")
            t = time.time() - start_time
            print("Elapsed time:", round((t)-(t%1))/60, "minutes\n")
            prev = round((count/load)*10)
    for j in range(len(y)):
        for keyword in y[j]:
            if listKeyword == keyword:
                if X[j] not in X_dict:
                    X_dict[X[j]] = {}
                if listKeyword not in X_dict[X[j]]:
                    X_dict[X[j]][listKeyword] = 1
                else:
                    X_dict[X[j]][listKeyword] += 1
    count += 1

10 % complete...
Elapsed time: 3.7333333333333334 minutes

20 % complete...
Elapsed time: 11.1 minutes

30 % complete...
Elapsed time: 18.316666666666666 minutes

40 % complete...
Elapsed time: 25.633333333333333 minutes

50 % complete...
Elapsed time: 32.86666666666667 minutes

60 % complete...
Elapsed time: 41.81666666666667 minutes

70 % complete...
Elapsed time: 988.5666666666667 minutes

80 % complete...
Elapsed time: 997.3666666666667 minutes

90 % complete...
Elapsed time: 1006.1333333333333 minutes

100 % complete...
Elapsed time: 1014.9666666666667 minutes



In [12]:
for i in list(X_dict):
    for j in list(X_dict[i]):
        if X_dict[i][j] == 1:
            del X_dict[i][j]

del X_dict["All Other"]
del X_dict["Other License"]
# Number of unique properties used in training set
print(len(X_dict))
# Will print FAT list of properties and list of words used with it. Just scroll past it.
# for i in X_dict:
#     print(i)
#     print(X_dict[i])
#     print("\n")

2704


In [28]:
property_tree = [
    ["avengers","marvel","marvel universe","marvel other","spider-man","iron man","guardian of the galaxy","guardians of the galaxy","captain america movie 3","thor movie 3",
     "avengers movie 3 part 1","captain america movie - do not use","Spider-Man Movie (2017)","Ultimate Spider-Man","Thor - Do Not Use"],
    ["disney princess","frozen","disney frozen","frozen 2","frozen ii","beauty and the beast","little mermaid","cinderella","tangled","sleeping beauty","snow white & the 7 dwarfs",
     "Princess and the Frog","aladdin","Pocahontas","Mulan"],
    ["cars the movie","cars","cars 2","cars 3"],
    ["mickey mouse","mickey & friends","minnie","mickey mouse club"],
    ["star wars","star wars: rogue one","star wars - other","the mandalorian","star wars: the force awakens (episode 7)","The Phantom Menace (Episode 1)","The Empire Strikes Back (Episode 5)",
    "Star Wars Sequels - Other","Clone Wars"],
    ["frozen","disney frozen","frozen 2","frozen ii"],
    ["winnie the pooh","winnie the pooh & friends"],
    ["disney fairies","fairies"],
    ["finding nemo/finding dory","finding dory","finding nemo"],
    ["lilo & stitch","lilo and stitch"],
    ["nightmare before christmas","nightmare before xmas"],
    ["big hero 6","big hero six"],
    ["Lion Guard","The Lion Guard"]
]
property_tree = [[x.lower() for x in sublist] for sublist in property_tree]

In [29]:
def testRandom(prodSplit, propList):
    correct = 0
    total = 0
    print("Go")
    ignored = 0
    for i in range(500):
        isCorrect = False
        index = random.randrange(len(propList))
        product = prodSplit[index]
#         print(product)
        top10 = {}
        top102 = {}
        followCheck = ""
        previousWord = ""
        for word in product:
#             print("Word: ", word)
            for property in X_dict:
                for keyword in X_dict[property]:
                    if word == keyword:
                        if property in top10:
                            if previousWord in X_dict[property]:
                                # TODO: Must change so doesnt multiply into total property score but into previous word's score
                                top10[property] *= X_dict[property][keyword]
                            else:
                                top10[property] += X_dict[property][keyword]
                        else:
                            top10[property] = X_dict[property][keyword]

                        if property in top102:
                            top102[property] += 1
                        else:
                            top102[property] = 1
#                         print("Property: ", property, X_dict[property][keyword])
            previousWord = word


        top10 = {k: v for k, v in sorted(top10.items(), key=lambda item: item[1], reverse=True)}
        top102 = {k: v for k, v in sorted(top102.items(), key=lambda item: item[1], reverse=True)}
        # ~40% of products are ignored
        if len(top10) < 2 or top10[next(iter(top10))]/top10[list(top10.keys())[1]] <= 20:
            ignored += 1
            continue
#         print(top10)
        total += 1
        if top10 != {}:
            if next(iter(top10)).lower() == propList[index].lower():
                correct += 1
                isCorrect = True
#                 print("Correct:", propList[index])
            else:
                for branch in property_tree:
                    if next(iter(top10)).lower() in branch and propList[index].lower() in branch:
                        correct += 1
                        isCorrect = True
#                         print("Correct:", next(iter(top10)), "&", propList[index])
                        break
                if not isCorrect:
                    print(product)
                    print("Incorrect:",next(iter(top10)),"Answer:", propList[index])
#                     print("Correct? (t/f/na):")
#                     x = input()
                    x = "f"
                    if x == "t":
                        correct += 1
                        isCorrect = True
                    elif x == "na":
                        total -= 1
#                     else:
#                         print(top10, "\n")
                
    print(ignored)
    print(correct/total)

In [111]:
def predict(prodSplit):
    done = 0
    count = 0
    load = 500
    prev = 0
    start_time = time.time()
    
    propList = [0]*500
    print("Running prediction...")
    for index in range(500):
        
        if round((count/load)*10) != prev:
            print(round((count/load)*10)*10, "% complete...")
            t = time.time() - start_time
            print("Elapsed time:", round((t)-(t%1))/60, "minutes\n")
            prev = round((count/load)*10)
            
        product = prodSplit[index+10000]
        top10 = {}
        top102 = {}
        followCheck = ""
        previousWord = ""
        for word in product:
            for property in X_dict:
                for keyword in X_dict[property]:
                    if word == keyword:
                        if property in top10:
                            if previousWord in X_dict[property]:
                                # TODO: Must change so doesnt multiply into total property score but into previous word's score
                                top10[property] *= X_dict[property][keyword]
                            else:
                                top10[property] += X_dict[property][keyword]
                        else:
                            top10[property] = X_dict[property][keyword]

                        if property in top102:
                            top102[property] += 1
                        else:
                            top102[property] = 1
            previousWord = word


        top10 = {k: v for k, v in sorted(top10.items(), key=lambda item: item[1], reverse=True)}
        top102 = {k: v for k, v in sorted(top102.items(), key=lambda item: item[1], reverse=True)}
        if len(top10) < 2 or top10[next(iter(top10))]/top10[list(top10.keys())[1]] <= 20:
            propList[index] = "Unmapped"
            done += 1
            continue
        if top10 != {}:
            propList[index] = next(iter(top10))
        else:
            print("None")
            propList[index] = "Unmapped"
            done += 1
    print("Mapping complete.")
    print((load-done)*100/load, "% mapped")            
    return propList

In [112]:
x = predict(mAmaSplit)

Running prediction...
Mapping complete.
62.6 % mapped


In [114]:
for i in range(20):
    j = random.randrange(len(x))
#     if x[j] == "Unmapped": continue
    print(mAmaSplit[j+10000])
    print(x[j],"\n")

['little', 'mermaid', 'ariel', 'glass', 'window', 'graphic', 't-shirt']
Disney Princess 

['marvel', 'boys', 'iron', 'man', 'swim', 'set', 'multicoloured', 'age', '7', '8', 'years']
Avengers 

['koin', 'club', 'gifts', '50p', 'shaped', 'coin', 'limited', 'edition', 'little', 'mermaid', 'c']
Disney Princess 

['winnie', 'pooh', 'friends', 'sketch', 'sweatshirt']
Winnie The Pooh & Friends 

['star', 'wars', '7th', 'birthday', 'card']
Star Wars 

['phone', 'case', 'huawei', 'p20', 'pro', 'incredible', 'hulk', 'bruce', 'banner', 'marvel', 'comics', '13']
Avengers 

['girl', 's', 'tinkerbell', 'silhouette', 'sweatshirt']
Unmapped 

['marvel', 'avengers', 'endgame', 'cast', 'symmetry', 'men', 's', 'hooded', 'sweatshirt', 'heather', 'grey']
Avengers 

['mulan-legende', 'einer', 'krieg', 'mo', '[blu-ray]', '[2009]']
Unmapped 

['christmas', 'cinderella', 'dancing', 'twirl', 'women', 's', 'sweatshirt', 'charcoal']
Disney Princess 

['frozen', 'ii', 'anna', 'elsa', 'olaf', 'believe', 'journey', 